<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/NLP/RNN_IMDB_Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  8952k      0  0:00:09  0:00:09 --:--:-- 16.4M


In [2]:
!tar -xf aclImdb_v1.tar.gz

In [3]:
!ls -l

total 82164
drwxr-xr-x 4 7297 1000     4096 Jun 26  2011 aclImdb
-rw-r--r-- 1 root root 84125825 Dec 28 02:31 aclImdb_v1.tar.gz
drwxr-xr-x 1 root root     4096 Dec 20 20:19 sample_data


In [4]:
!rm -r aclImdb/train/unsup

In [5]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [6]:
import os, pathlib, shutil, random

In [7]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

In [8]:
for category in ("pos", "neg"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_of_val_files = int(len(files) * 0.2)

  val_files = files[-num_of_val_files:]

  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

In [9]:
import tensorflow as tf
from tensorflow import keras

In [10]:
batch_size = 32

In [11]:
train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size = batch_size)

Found 20000 files belonging to 2 classes.


In [12]:
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)

Found 5000 files belonging to 2 classes.


In [13]:
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size = batch_size)

Found 25000 files belonging to 2 classes.


In [14]:
for inputs, targets in train_ds:
  print(inputs[0])
  print(targets[0])
  print(inputs.shape)
  print(targets.shape)
  print(inputs.dtype)
  print(targets.dtype)
  data = inputs[0]
  break

tf.Tensor(b'This was my first, and probably the last Angelopoulos movie. I was eager to get into it, as it featured Mastroianni, one of my favorite actors and was a film By Theo, of whom I\'ve heard a lot. The opening was promising, a long shot over a jeep of soldiers across the Albanian-Greek border. OK! but that was all. Nothing left. The movie had big holes and I don\'t know which to mention first. The main plot of the story is revealed to the journalist by the old woman. during a long walk. It\'s like a 15 minutes monologue, killing the action and viewers patience, nothing happening on screen for 15 or even 20 minutes, apart this old lady telling a story. All that is presumed to be shown through action, was simply told to the camera by the old lady. In a moment, the equippe of TV was heading to the bar. They turn the corner and immediately the winter begins! Probably, shot in different days, continuity leaked. A lot of problems with the story-telling, it went from absurd to irratio

In [15]:
from keras.layers import TextVectorization

In [24]:
text_vectorization = TextVectorization(output_mode="multi_hot", max_tokens=2000)

In [17]:
text_ds = train_ds.map(lambda x, y: x)

In [36]:
target_train_ds = train_ds.map(lambda x, y: y)

In [37]:
target_train_ds

<MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>

In [29]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [30]:
type(text_ds)

tensorflow.python.data.ops.dataset_ops.MapDataset

In [31]:
len(train_ds)

625

In [32]:
len(text_ds)

625

In [35]:
text_ds

<MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [34]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [25]:
text_vectorization.adapt(inputs)

In [26]:
ds = text_vectorization.get_vocabulary()

In [27]:
print(ds)

['[UNK]', 'the', 'and', 'of', 'a', 'to', 'is', 'in', 'as', 'it', 'that', 'this', 'film', 'i', 'was', 'for', 'with', 'his', 'be', 'on', 'but', 'br', 'one', 'he', 'not', 'movie', 'its', 'by', 'all', 'are', 'her', 'you', 'from', 'has', 'an', 'who', 'at', 'just', 'have', 'about', 'would', 'story', 'or', 'which', 'very', 'she', 'films', 'what', 'so', 'if', 'even', 'into', 'really', 'never', 'more', 'first', 'when', 'they', 'character', 'like', 'were', 'tarzan', 'only', 'no', 'much', 'good', 'time', 'their', 'other', 'had', 'can', 'well', 'there', 'some', 'scene', 'movies', 'most', 'lot', 'best', 'see', 'could', 'back', 'also', 'while', 'than', 'should', 'seen', 'rob', 'points', 'out', 'my', 'me', 'man', 'do', 'been', 'any', 'world', 'watch', 'up', 'too', 'through', 'then', 'them', 'take', 'know', 'him', 'here', 'get', 'before', 'after', 'zizek', 'your', 'though', 'think', 'same', 'roth', 'poor', 'jane', 'interesting', 'great', 'death', 'course', 'actually', 'acting', 'young', 'yet', 'why', 

In [28]:
len(ds)

2000